In [19]:
import xarray as xr
import pandas as pd
import numpy as np

def calculate_rmse(xarray_file_path, excel_file_path, lat_col='lat', lon_col='lon', mb_mea_col='MB_MEA', mb_mod_col='MB_MOD'):
    """
    计算 MB_MEA 和 MB_MOD 之间的均方根误差 (RMSE)。

    参数:
    - xarray_file_path: str, NetCDF 文件的路径。
    - excel_file_path: str, Excel 文件的路径。
    - lat_col: str, DataFrame 中纬度列的名称，默认为 'lat'。
    - lon_col: str, DataFrame 中经度列的名称，默认为 'lon'。
    - mb_mea_col: str, DataFrame 中 MB_MEA 列的名称，默认为 'MB_MEA'。
    - mb_mod_col: str, DataFrame 中 MB_MOD 列的名称，默认为 'MB_MOD'。

    返回:
    - df: pandas DataFrame, 包含提取的 MB_MOD 值的 DataFrame。
    - rmse: float, MB_MEA 和 MB_MOD 之间的 RMSE 值。
    """
    # 1. 加载 xarray 数据集
    data = xr.open_dataset(xarray_file_path)

    # 2. 加载 Excel 文件
    df = pd.read_excel(excel_file_path)

    # 3. 对 MB_MEA 列进行单位转换（从毫米转换为米）
    df[mb_mea_col] = df[mb_mea_col] / 1000

    # 4. 创建一个新列来存储从 xarray 数据集中提取的 MB_MOD 值
    df[mb_mod_col] = None

    # 5. 遍历每一行，提取对应经纬度的 MB_MOD 值
    for index, row in df.iterrows():
        # 提取当前行的经纬度
        lat = row[lat_col]
        lon = row[lon_col]
        
        # 使用 .sel() 方法找到最接近的经纬度点，并沿时间维度求和
        mb_mod_value = data.sel(lat=lat, lon=lon, method='nearest').sum(dim='time')['MB'].values
        
        # 将提取的值存储到新列中
        df.at[index, mb_mod_col] = mb_mod_value

    # 6. 计算 MB_MEA 和 MB_MOD 之间的均方根误差 (RMSE)
    difference = df[mb_mea_col] - df[mb_mod_col]  # 计算差值
    squared_difference = difference ** 2  # 计算平方差
    rmse = np.sqrt(squared_difference.mean())  # 计算 RMSE

    # 7. 返回结果
    return df, rmse


# 使用函数
xarray_file_path = "/home/rice/cosipy/data/output/kqgr_0.8_20220607-20220610.nc"
excel_file_path = '/home/rice/cosipy/data/2122年度处理后的物质能量平衡.xlsx'

df, rmse = calculate_rmse(xarray_file_path, excel_file_path)

# 输出结果
print("DataFrame with extracted MB_MOD values:")
print(df)
print(f"\nRMSE between MB_MEA and MB_MOD: {rmse}")

DataFrame with extracted MB_MOD values:
         lat       lon  MB_MEA            H                 MB_MOD
0   29.86499  90.19854  -2.208  5556.139160  -0.018942695146065495
1   29.86693  90.19571  -2.361  5565.989258                    0.0
2   29.86700  90.19577  -2.322  5562.896484                    0.0
3   29.86655  90.19531  -2.211  5576.324707  -0.017275051177316484
4   29.86645  90.19459  -1.932  5587.385742  -0.017785613111350216
5   29.86412  90.19699  -1.917  5592.370117   -0.01618129990599302
6   29.86569  90.19568  -2.133  5587.401367  -0.017369571903518188
7   29.86420  90.19486  -1.581  5613.258789  -0.015516156464953973
8   29.86304  90.19528  -1.683  5630.861816  -0.013837538985459912
9   29.86299  90.19367  -1.641  5640.732422  -0.014069252951210188
10  29.86290  90.19365  -1.893  5642.345215  -0.014069252951210188
11  29.86237  90.19313  -1.524  5651.476562  -0.013696242420649882
12  29.86209  90.19285  -1.446  5655.983887  -0.013775039067012594
13  29.86097  90.19300